# yolov5葡萄数量预测模型训练

### 拉取代码

In [1]:
%cd yolov5

e:\SRP\grape_yolov5\yolov5


### 配置环境

In [ ]:
%pip install -r requirements.txt

在训练过程中使用的数据增强策略可以在使用的训练配置文件中设置，我们使用的是yolov7/data/hyp.scratch.tiny.yaml：

In [ ]:
%pycat data/hyps/hyp.scratch-high.yaml

## 训练模型

### check anchor
重新计算anchor

葡萄数目预测anchor计算

In [2]:
import utils.autoanchor as autoAC
 
# 对数据集重新计算 anchors
new_anchors = autoAC.kmean_anchors('./data/grape_number_prediction.yaml', 6, 480, 5.0, 1000, False)
print(new_anchors)

d:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Scanning ..\datasets\train.cache... 774 images, 0 backgrounds, 0 corrupt: 100%|██████████| 774/774 00:00
AutoAnchor: Running kmeans for 6 anchors on 28400 points...
AutoAnchor: Evolving anchors with Genetic Algorithm: fitness = 0.8733: 100%|██████████| 1000/1000 00:02
AutoAnchor: thr=0.20: 1.0000 best possible recall, 4.91 anchors past thr
AutoAnchor: n=6, img_size=640, metric_all=0.574/0.873-mean/best, past_thr=0.677-mean: 23,24, 32,32, 39,40, 46,47, 58,59, 275,350


[[     22.876      23.957]
 [     31.837      32.303]
 [     39.005      39.773]
 [     46.383      47.215]
 [     58.382      58.948]
 [     275.14       350.2]]


### 查看模型配置文件

In [ ]:
%pycat models/yolov5s.yaml

In [ ]:
!python models/yolo.py --cfg yolov5s.yaml

### 训练模型

葡萄串检测模型训练：

In [ ]:
!python train.py --workers 12 --batch-size 16 --cfg models/yolov5s_grape.yaml --epochs 10 --device 0 --weights yolov5s.pt --hyp data/hyps/hyp.scratch-low.yaml --data data/grape.yaml --img 640

葡萄数目检测模型训练：

In [ ]:
!python train.py --workers 12 --batch-size 4 --cfg models/yolov5s_num_predection.yaml --epochs 500 --device 0 --weights yolov5s.pt --hyp data/hyps/hyp.scratch-high.yaml --data data/grape_number_prediction.yaml --img 640

## 查看结果

训练结束，可以看到run/train文件夹下保存了训练结果，加载best.pt查看推理结果：

### 图片灰度处理

In [5]:
from PIL import Image

# 打开图像文件
image = Image.open('../datasets/grape/1970_01_01_18_40_IMG_0688.JPG')

# 将图像转换为灰度图像
gray_image = image.convert('L')

# 保存灰度图像
gray_image.save('../1970_01_01_18_40_IMG_0688.JPG')

In [ ]:
!python detect.py --weights best.pt --source ../datasets/images/test --img-size 640 --device 0 --data yolov5/data/grape.yaml --save-txt

In [2]:
!python detect.py --weights ShuffleNet_Yolov5_int8.onnx --source detect_image --img-size 640 --device 0  --save-txt  --save-conf

detect: weights=['yolov5s_MobileNetv3_small_int8.onnx'], source=detect_image, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2024-2-27 Python-3.9.13 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Loading yolov5s_MobileNetv3_small_int8.onnx for ONNX Runtime inference...
image 1/4 E:\SRP\grape_yolov5\yolov5\detect_image\P0040.jpg: 640x640 (no detections), 78.8ms
image 2/4 E:\SRP\grape_yolov5\yolov5\detect_image\P0041.jpg: 640x640 (no detections), 52.0ms
image 3/4 E:\SRP\grape_yolov5\yolov5\detect_image\TEST_1.jpg: 640x640 (no detections), 47.4ms
image 4/4 E:\SRP\grape_yolov5\yolov5\detect_image\TEST_2.jpg: 640x640 

可以看到结果保存在runs/detect/exp?/*.jpg，查看结果：

In [ ]:
import cv2
from  matplotlib import pyplot as plt
%matplotlib inline

img = cv2.imread("runs/detect/exp5/G0006.jpg")
plt.imshow(img[:,:,::-1])
plt.show()

## 测试训练的权重

### 正常测试

In [3]:
!python val.py --workers 12 --weights best.pt --batch-size 16 --device cpu --data data/grape_number_prediction.yaml --img 480

^C


val: data=data/grape_number_prediction.yaml, weights=['best.pt'], batch_size=16, imgsz=480, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=cpu, workers=12, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2024-2-27 Python-3.9.13 torch-2.1.2+cu121 CPU

Fusing layers... 
YOLOv5s_tiny summary: 169 layers, 150126 parameters, 0 gradients, 1.0 GFLOPs

val: Scanning E:\SRP\grape_yolov5\datasets\val.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 00:00
val: Scanning E:\SRP\grape_yolov5\datasets\val.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 00:00

                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0%|          | 0/3 00:00
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  33%|███▎      | 1/3 00:16
               

### 模型剪枝

In [ ]:
from utils.torch_utils import prune

prune('best.pt', 0.3)

## 导出权重文件为其他格式

In [3]:
!python export.py --weights ./best.pt --include onnx --device cpu --optimize

export: data=E:\SRP\grape_yolov5\yolov5\data\coco128.yaml, weights=['./best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=True, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5  2024-2-27 Python-3.9.13 torch-2.1.2+cu121 CPU

Fusing layers... 
YOLOv5s_MobileNetv3_small summary: 286 layers, 2562853 parameters, 0 gradients, 4.4 GFLOPs

PyTorch: starting from best.pt with output shape (1, 25200, 7) (5.3 MB)

ONNX: starting export with onnx 1.14.1...
ONNX: export success  2.2s, saved as best.onnx (10.2 MB)

Export complete (2.6s)
Results saved to E:\SRP\grape_yolov5\yolov5
Detect:          python detect.py --weights best.onnx 
Validate:        python val.py --weights best.onnx 
PyTorch Hub:     model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.onnx')  
Visual